In [ ]:
import pandas as pd

In [ ]:
# %%bash
# git clone https://github.com/Jaehun-Kim22/SCIC_Project.git


In [ ]:
## loads excel
xls = pd.ExcelFile('../assets/data.xlsx')
training_data = pd.read_excel(xls, '학습데이터')
priority = pd.read_excel(xls, '우선순위')
data_without_ground_truth = training_data.values[:, 2]

In [ ]:
print(training_data, priority)
print(data_without_ground_truth)


In [ ]:
## loads excel
# xls = pd.ExcelFile('SCIC_Project/assets/data.xlsx')
# training_data = pd.read_excel(xls, '학습데이터')
# priority = pd.read_excel(xls, '우선순위')
data_without_ground_truth = training_data.values[:, 2]
# !pip install konlpy
from konlpy.tag import *
import copy
import time
kkma = Kkma()
okt = Okt()
kom = Komoran()


print(training_data.values[:, 2][:20])


def perform_spacing_Okt(text):
  tagged = okt.pos(text)
  corrected = ""
  for i in tagged:
      if i[1] in ('Josa', 'PreEomi', 'Eomi', 'Suffix', 'Punctuation', 'Unknown'):
          corrected += i[0]
      else:
          corrected += " "+i[0]
  if corrected[0] == " ":
      corrected = corrected[1:]
  return corrected


def perform_Kkma(text):
  temp_okt = perform_spacing_Okt(text)
  temp_kkma = kkma.sentences(temp_okt)
  # temp_kkma = kkma.sentences(text)
  if [text] != temp_kkma:
    new_string_to_add = ''
    new_string_to_add += ' '.join(temp_kkma)
    return new_string_to_add
  else:
    return text
s = time.time()
training_data_copy_kkma = copy.deepcopy(training_data)


cnt = 0
column_added_kkma = []
for text in training_data['발화']:
  column_added_kkma.append(perform_Kkma(text))
  cnt += 1
training_data_copy_kkma['Kkma'] = column_added_kkma
print(training_data_copy_kkma.values[:, 10][:20])
e = time.time()
print(e - s)